In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

import psycopg2
import copy

from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [11]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

In [3]:
def get_def_info(word, from_lang, to_lang, multisence_dict, print_ouput = False):
    multisence_dict[word] = {'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                if print_ouput: print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                if print_ouput:print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    if print_ouput:print("ENG_DEFIN_EX", ex.text.strip())
                if print_ouput:print(current_definition)
                multisence_dict[word]['lemma_branch'].append(current_definition)
                current_definition = {}
                if print_ouput:print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_def_info("book", "english","russian",sence_dict)


In [4]:
sence_dict['book']


{'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
   'local_word': 'книга',
   'examples': ['a book about animals']},
  {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
   'local_word': 'книжечка',
   'examples': []},
  {'definition': 'a set of pages fastened together in a cover and used for writing on',
   'local_word': 'записная книжка',
   'examples': ['an address book']},
  {'definition': 'to arrange to use or do something at a particular time in the future',
   'local_word': 'бронировать',
   'examples': ['to book a ticket/hotel room',
    "We've booked a trip to Spain for next month.",
    'Sorry, the hotel is fully booked (= has no more rooms).']},
  {'definition': 'to officially accuse someone of a crime',
   'local_word': 'заводить дело',
   'examples': ['Detectives booked him for resisting arrest.']},
  {'definition': 'If a sports official books you, they write an official record of some

In [5]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def nltk_lemmatize(word):
    pos = pos_tag(word_tokenize(word))[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    if wordnet_pos:
        lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
    else:
        lemma = lemmatizer.lemmatize(word)
    return lemma

nltk_lemmatize("having")

'have'

In [14]:
def get_sencewords_and_examples_reverso(eng_wordform, from_lang, to_lang, print_output = False):
    wordform_localsencewords_examples_dict = {}
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + eng_wordform 
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    first_string_passed = False
    translations_variants = []
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            if print_output:print(sence_word)
            wordform_localsencewords_examples_dict[sence_word] = []
            translations_variants.append(sence_word)
        first_string_passed = True
    time.sleep(random.uniform(0.4,0.8))
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,1.0))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        if print_output:
            print("========================================")
            print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + eng_wordform + "\w{0,1}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                wordform_localsencewords_examples_dict[transl_word].append(example)
                if print_output: print(example)
    return wordform_localsencewords_examples_dict
w_locsence_ex_dict = get_sencewords_and_examples_reverso('having',"английский","русский", print_output = True )   
    

наличие
после
имеющих
имеющие
чтобы
имея
весело
необходимости
того
оказывает
есть
имеют
имеет
поскольку
оказывают

EXAMPLES FOR SENCE  наличие 


EXAMPLES FOR SENCE  после 


EXAMPLES FOR SENCE  имеющих 

Such policy completely ensures enrolment in the higher education system of students having such social status.
Article 120 of the Labour Code fixes the duration of basic leave for employees recognized as having rendered special service to Azerbaijan.
Such persons should be coded as having no fixed place of work.

EXAMPLES FOR SENCE  имеющие 

The President was given the authority to issue decrees having the force of law.
The Indian Supreme Court has interpreted such provisions as having real significance.
Grant me, Divine Father, the delegation of having power over malevolent spirits.

EXAMPLES FOR SENCE  чтобы 


EXAMPLES FOR SENCE  имея 

Not having a rope, you can easily lose each other.
It's like I'm having some kind of a breakdown.
They're just having a big meeting about next fal

In [7]:
w_locsence_ex_dict

{'наличие': [],
 'после': [],
 'имеющих': ['Such policy completely ensures enrolment in the higher education system of students having such social status.',
  'Article 120 of the Labour Code fixes the duration of basic leave for employees recognized as having rendered special service to Azerbaijan.',
  'Such persons should be coded as having no fixed place of work.'],
 'имеющие': ['The President was given the authority to issue decrees having the force of law.',
  'The Indian Supreme Court has interpreted such provisions as having real significance.',
  'Grant me, Divine Father, the delegation of having power over malevolent spirits.'],
 'чтобы': [],
 'имея': ['Not having a rope, you can easily lose each other.',
  "It's like I'm having some kind of a breakdown.",
  "They're just having a big meeting about next fall's chorus program.",
  'Indeed, having a shopping cart is the internet marketing strategy of the new era.',
  "And it's amazing how having one nice thing actually made us a 

In [8]:

l = ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']
word = 'забронировать'
def look_for_sublemma_word(wordform, lemmas_list):
    if isinstance(lemmas_list, list) or isinstance(lemmas_list, set):
        for lemma in lemmas_list:
            if len(lemma.split()) == 1:
                if wordform in lemma or lemma in wordform:
                    #print(wordform)
                    return lemma
        return None
    elif isinstance(lemmas_list, str):
        if len(lemmas_list.split()) == 1:
            if wordform in lemmas_list or lemmas_list in wordform:
                return lemma
            else:
                return None
    else:
        return None #"no type" + str(type(lemmas_list))

look_for_sublemma_word(word, l) 
            

'бронировать'

In [9]:
def merge_lemma_context_vs_oxford_examples(eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    oxford_sensewords_list = []
    oxford_lemmas_examples_pointers_dict = {}
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
        oxford_lemmas_examples_pointers_dict[definition_element['local_word']] = definition_element['examples']
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    for context_local_lemma in local_wordsence_examples_dict.keys():
        if print_output: print("\nNOWPROCESSING", context_local_lemma)
        orig_intersection_lemma = look_for_sublemma_word(context_local_lemma, oxford_sensewords_list)
        if not orig_intersection_lemma:
            if print_output: print(context_local_lemma, "\nno such lemma in orig dict")
            pass
        elif len(local_wordsence_examples_dict[context_local_lemma]) == 0:
            if print_output: print(context_local_lemma, "\ndoes not have examples with english wodform")
            pass
        else:
            #general_dict[eng_lemma]['lemma_branch'][context_local_lemma]['examples'].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print(oxford_lemmas_examples_pointers_dict[orig_intersection_lemma])
            oxford_lemmas_examples_pointers_dict[orig_intersection_lemma].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print("add",local_wordsence_examples_dict[context_local_lemma][:3] )
dct = copy.deepcopy(sence_dict)
merge_lemma_context_vs_oxford_examples("book",w_locsence_ex_dict, dct, print_output = True)        
dct


WILL LOOK THROUGH LOCAL WORDFORMS ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']

NOWPROCESSING наличие
наличие 
no such lemma in orig dict

NOWPROCESSING после
после 
no such lemma in orig dict

NOWPROCESSING имеющих
имеющих 
no such lemma in orig dict

NOWPROCESSING имеющие
имеющие 
no such lemma in orig dict

NOWPROCESSING чтобы
чтобы 
no such lemma in orig dict

NOWPROCESSING имея
имея 
no such lemma in orig dict

NOWPROCESSING весело
весело 
no such lemma in orig dict

NOWPROCESSING необходимости
необходимости 
no such lemma in orig dict

NOWPROCESSING того
того 
no such lemma in orig dict

NOWPROCESSING оказывает
оказывает 
no such lemma in orig dict

NOWPROCESSING есть
есть 
no such lemma in orig dict

NOWPROCESSING имеют
имеют 
no such lemma in orig dict

NOWPROCESSING имеет
имеет 
no such lemma in orig dict

NOWPROCESSING поскольку
поскольку 
no such lemma in orig dict

NOWPROCESSING оказывают
оказывают 
no such lemma in orig 

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': []},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hotel room',
     "We've booked a trip to Spain for next month.",
     'Sorry, the hotel is fully booked (= has no more rooms).']},
   {'definition': 'to officially accuse someone of a crime',
    'local_word': 'заводить дело',
    'examples': ['Detectives booked him for resisting arrest.']},
   {'definition': 'If a sports official books you, they write 

In [21]:
def merge_locsence_ex_vs_lemma_structure(eng_wordform, eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    general_dict[eng_lemma]['wordforms_branch'][eng_wordform] = []
    oxford_sensewords_list = []
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    successfull_merge_count = 0
    for local_wordform in local_wordsence_examples_dict.keys():
        p = morph.parse(local_wordform)[0]
        local_lemma = p.normal_form
        if print_output: print("\nNOWPROCESSING", local_wordform, local_lemma, look_for_sublemma_word (local_lemma, oxford_sensewords_list))
        if look_for_sublemma_word (local_wordform, oxford_sensewords_list):
            if print_output: print(local_wordform, "\nis original lemma itself")
            pass
        elif len(local_wordsence_examples_dict[local_wordform]) == 0:
            if print_output: print(local_wordform, "\ndoes not have examples with english wodform")
            pass
        else:
            if print_output: print(local_wordform, "\nno match but no limitations")
            local_wordform_dict = {"sence_wordform_local":local_wordform,"context":local_wordsence_examples_dict[local_wordform][:3]}
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform].append(local_wordform_dict)
        """
        elif look_for_sublemma_word (local_lemma, oxford_sensewords_list) == True:
            if print_output: print(local_wordform, "\nlocal_wordform to local lemma or viceversa match")
            successfull_merge_count += 1
            local_wordform_dict = {"sence_wordform_local":local_wordform,
                                   "context":[]}
            for definition_element in general_dict[eng_lemma]['lemma_branch']:
                
                if look_for_sublemma_word (local_lemma, definition_element['local_word']):
                    local_wordform_examples = copy.deepcopy(definition_element['examples'])
                    local_wordform_dict['context'] = local_wordform_examples
                    try:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[local_wordform][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[local_wordform][:3])
                    except:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform].append(local_wordform_dict)
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
        """
        
    if print_output:
        if successfull_merge_count == 0: print("NO LOCAL INTERSECTIONS FOR WORDWORM", eng_wordform)
    if len(general_dict[eng_lemma]['wordforms_branch'][eng_wordform]) == 0:
        del general_dict[eng_lemma]['wordforms_branch'][eng_wordform] 
        
        
#merge_locsence_ex_vs_lemma_structure("booking","book",w_locsence_ex_dict, big_diCKt, print_output = True)        
#big_diCKt

In [ ]:
time.now()

In [26]:
def process_lemma_and_forms(word, dictionary, show_words = False):
    moment_1 = time.time()
    get_def_info(word, "english","russian",dictionary)
    moment_2 = time.time()
    print("get_def_info",moment_2 - moment_1)
    
    lemma_sence_ex_dict = get_sencewords_and_examples_reverso(word,"английский","русский", print_output = False )
    moment_3 = time.time()
    print("lemma_sence_ex_dict",moment_3 - moment_2)
    
    merge_lemma_context_vs_oxford_examples(word,lemma_sence_ex_dict, dictionary, print_output = False) 
    moment_4 = time.time()
    print("merge_lemma_context_vs_oxford_examples",moment_4 - moment_3)
    
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    moment_5 = time.time()
    print("word request to lldb",moment_5 - moment_4)
    
    req_res = cursor.fetchone()
    if not req_res:
        if show_words == True: print(word, " is not in db")
        return 0
    else:
        #print(a[0])
        word_id = req_res[0]
        if show_words == True: print(req_res[2])
    
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    moment_6 = time.time()
    print("word forms request to lldb",moment_6 - moment_5)
    for wordform in cursor:
        if show_words == True: print(wordform[0])
        wordform_lemma = nltk_lemmatize(wordform[0])
        if wordform_lemma == word:
            if show_words == True: print("will process", wordform[0])
            start = time.time()
            local_wordforms_sence_examples_dict = get_sencewords_and_examples_reverso(wordform[0],"английский", "русский")  
            merge_locsence_ex_vs_lemma_structure(wordform[0],word, local_wordforms_sence_examples_dict,dictionary ,
                                                                                 print_output= False)
            end = time.time()
            print("wordform iteration", end - start)
            #если после этого узел englsih_wordform оказывается пустым списком значит reverso не нашел нормальной словесной единицы для этого
            #то есть слово скорее всгео херовое как показала практика типа haveing haved
        else:
            if show_words == True: print("will not process", wordform[0], "lemma is",wordform_lemma )
        #get_def_info(wordform[0], "english","russian",dictionary,is_word_form = True, original_word = word )
        
big_diCKt = {}        
process_lemma_and_forms("book",big_diCKt)
big_diCKt

get_def_info 4.0450239181518555
lemma_sence_ex_dict 19.71959114074707
merge_lemma_context_vs_oxford_examples 0.000225067138671875
word request to lldb 1.325551986694336
word forms request to lldb 0.011990070343017578
wordform iteration 10.0308837890625
wordform iteration 4.740480899810791
wordform iteration 18.385943174362183


{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     'Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
     'The book aimed to sensitize children to the different aspects of poverty.',
     'The book is available on the OHCHR web site.']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': ['Have you got an autograph book?',
     'Where is your book?',
     'You taking Little League book like you did last year?']},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hot

In [ ]:
import json
with open("one_word_transalte.json", "w") as f:
    json.dump(big_diCKt, f, ensure_ascii=False, indent = 4)

In [ ]:
big_diCKt

In [15]:
import pandas as pd
from tqdm import tqdm

In [16]:
test_words_db = pd.read_csv("test_words.csv")
test_words = list(test_words_db['word'])

In [18]:
a = {'f':[]}

In [20]:
del a['f']
a

{}

In [27]:
global_dict = {}
for word in tqdm(test_words[101:]):
    process_lemma_and_forms(word.lower(), global_dict)




  0%|          | 0/99 [00:00<?, ?it/s]

get_def_info 4.092491149902344
lemma_sence_ex_dict 3.1477370262145996
merge_lemma_context_vs_oxford_examples 0.00018477439880371094





  1%|          | 1/99 [00:09<14:42,  9.01s/it]

word request to lldb 1.756822109222412
word forms request to lldb 0.011679887771606445
get_def_info 3.841831922531128
lemma_sence_ex_dict 2.5514490604400635
merge_lemma_context_vs_oxford_examples 0.00018405914306640625





  2%|▏         | 2/99 [00:16<13:48,  8.55s/it]

...

wordform iteration 19.242025136947632
get_def_info 3.9408249855041504
lemma_sence_ex_dict 18.176921844482422
merge_lemma_context_vs_oxford_examples 0.0006041526794433594
word request to lldb 1.0007801055908203
word forms request to lldb 0.03246188163757324
wordform iteration 0.9461040496826172





 99%|█████████▉| 98/99 [52:27<00:38, 38.29s/it]

wordform iteration 1.175771951675415
get_def_info 4.100193023681641
lemma_sence_ex_dict 0.9635319709777832
merge_lemma_context_vs_oxford_examples 0.0003509521484375





100%|██████████| 99/99 [52:33<00:00, 28.64s/it]

word request to lldb 1.0372672080993652


In [25]:
with open("big_dict_attempt_v3.json", "w") as f:
    json.dump(global_dict, f, ensure_ascii=False, indent = 4)